In [ ]:
!pip install BeautifulSoup4
import requests 
from bs4 import BeautifulSoup

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
# XPATH_LIST_CONGRESSMEN = '//a[@class="conginfo"]/text()'
# XPATH_LIST_URLCONGINFO = '//a[@class="conginfo"]/@href'
# #in previous url
# XPATH_REPRESENT = '//p[@class="representa"]/span[@class="value"]/text()'
URLCONGRESSMEN = 'https://www.congreso.gob.pe/pleno/congresistas/'

In [ ]:
#GET ALL NAMES AND URL'S
PLENO_PAGE = requests.get(URLCONGRESSMEN).text
PLENO_SOUP = BeautifulSoup(PLENO_PAGE, 'lxml')
LIST_PLENO = PLENO_SOUP.find_all(class_='conginfo')
LIST_CONGRESSMEN_NAMES = [i.text for i in LIST_PLENO]
LIST_URLCONGINFO = [i['href'] for i in LIST_PLENO]

In [ ]:
#GETTING REGIONS
LIST_REGIONES = []
LIST_VOTACION = []
for url in LIST_URLCONGINFO:
  CONG_PAGE = requests.get(URLCONGRESSMEN+url).text
  CONG_SOUP = BeautifulSoup(CONG_PAGE, 'lxml')
  LIST_REGIONES.append(CONG_SOUP.find('p', class_='representa').find(class_='value').text)
  LIST_VOTACION.append(CONG_SOUP.find('p', class_='votacion').find(class_='value').text)


In [ ]:
import pandas as pd
Congress = pd.DataFrame({"Congresista":LIST_CONGRESSMEN_NAMES, "Región":LIST_REGIONES, "Votación":LIST_VOTACION})
Congress['Comisiones'] = [[] for i in range(len(Congress['Congresista']))]

Primero se getea la pagina, después se convierte en sopa con BeautifulSoup (usando el atributo text y el atributo 'lxml') y después se usa el método find_all para buscar las cosas, después se usa el metodo get en cada elemento de la lista obtenida para obtener atributos, se puede usar el metodo get_text para obtener el texto

In [ ]:
urlcomisiones = 'https://www.congreso.gob.pe/CuadrodeComisiones/'
P_comisiones = requests.get(urlcomisiones)
P_comisiones.status_code


200

In [ ]:
soup_comisiones = BeautifulSoup(P_comisiones.text, 'lxml')
tags_comisiones = soup_comisiones.find_all('a', target='_blank')
comisiones = tags_comisiones[:24]

In [ ]:
links_comisiones = [i.get('href') for i in comisiones]
nombre_comisiones = [i.text for i in comisiones]


In [ ]:
n=0
COMISIONES_MIEMBROS_CARGOS = []
for URL_COMISION in links_comisiones:
  nonrawpage = requests.get(URL_COMISION+'/sobrelacomision/integrantes/')
  rawsoup = BeautifulSoup(nonrawpage.text, 'lxml')
  try:
    URL_COMISIONreal = rawsoup.find('iframe', attrs={'name':'ventana02'}).get('src')
  except:
    print("la interfaz no tiene los integrantes. Comision: "+nombre_comisiones[links_comisiones.index(URL_COMISION)])
  PaginaComision = requests.get(URL_COMISIONreal)
  SoupComision = BeautifulSoup(PaginaComision.text, 'lxml')
  Miembros = [i.get_text() for i in SoupComision.find_all('a', attrs={'class':'conginfo'})]
  Cargos = [ i.get_text() for i in SoupComision.find_all('span', attrs={'class':'partidolist'})][1::2]
  COMISIONES_MIEMBROS_CARGOS.append({Miembros[i]:[i[0] for i in Cargos][i] for i in range(len(Miembros))})
  n+=1


In [ ]:
for zip in COMISIONES_MIEMBROS_CARGOS:
  for (name, cargo) in zip.items():
        listindex=([nombre_comisiones[COMISIONES_MIEMBROS_CARGOS.index(zip)], cargo])
        Congress['Comisiones'][Congress.index[Congress['Congresista']==name].tolist()[0]].append(listindex)


In [ ]:
#remember that response is the variable that contains URLCONGRESSMEN
soup_partido = BeautifulSoup(PLENO_PAGE, 'lxml')
partidos = [i.get_text() for i in soup_partido.find_all('span', attrs={'class':'partidolist'})]
Congress['Bancada'] = partidos


In [ ]:
WIKI_POP_URL = 'https://es.wikipedia.org/wiki/Anexo:Departamentos_del_Per%C3%BA_por_poblaci%C3%B3n'
wiki_page = requests.get(WIKI_POP_URL).text
wiki_soup = BeautifulSoup(wiki_page, 'lxml')

In [ ]:
list(wiki_soup.tbody.children)[10].find_all('a')[0].text

'Arequipa'

In [ ]:
POP = {'Región':[], 'Población':[]}
for i in range(4,54,2):
    row = list(wiki_soup.tbody.children)[i]
    raw_number = row.find_all(align='right')[1].text
    number = int(raw_number.replace('\xa0', ''))
    region = row.find_all('a')[0].text
    POP['Región'].append(region)
    POP['Población'].append(number)
POP_DF =  pd.DataFrame(POP)
POP_DF = pd.concat([POP_DF, pd.DataFrame([['Lima Provincias', 594613], ['PEX', 457891]], 
                                        columns=['Región', 'Población'])]).reset_index(drop=True)
POP_DF

,Región,Población
0,Amazonas,379384
1,Áncash,1083519
2,Apurímac,405759
3,Arequipa,1382730
4,Ayacucho,616176
5,Cajamarca,1341012
6,Callao,994494
7,Cusco,1205527
8,Huancavelica,347639
9,Huánuco,721047


In [ ]:
Congress = Congress.replace('Ancash', 'Áncash')
Congress = Congress.replace(['Peruanos Residentes en el Extranjero', 'Lima Peruanos en el exterior'], 'PEX')
POB_CONG = Congress.merge(POP_DF, on='Región', how='left')
POB_CONG['Votación'] = POB_CONG['Votación'].apply(lambda x:int(x.replace(',', '')))

In [ ]:
POB_CONG['Representatividad'] = POB_CONG['Votación']/POB_CONG['Población']

CONG = POB_CONG.drop(['Población'], axis=1)

In [ ]:
# Congress.to_csv('Congress.csv', index=False)

In [ ]:
CONG.to_csv('CONG.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=496b7f65-72e8-4f99-acc9-201d5fae973f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>